In [1]:
from webull import paper_webull, endpoints # for real money trading, just import 'webull' instead
from webull.streamconn import StreamConn
import paho.mqtt.client as mqtt
import trendln
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import datetime
import sched
import requests
import time
import io
import os
from os import listdir
from os.path import isfile, join
from day_trade import login_webull
pd.set_option('display.max_rows', 1000)
import json

In [3]:
import matplotlib.pyplot as plt

plt.close('all')

In [2]:
loginInfo = json.load(open('token.txt'))

In [4]:
#Vars
symbol = None
period = None
timeframe = None
hist = []

In [5]:
hist = None
support = 0
resistance = 0
enteredTrade = False
s = sched.scheduler(time.time, time.sleep)

In [3]:
wb = login_webull('mcian91@email.com')

In [7]:
#Draw Chart
def drawChart(hist, update):
    global support
    global resistance
    global symbol
    try :
        mins, maxs = trendln.calc_support_resistance((hist[-1000:].low, hist[-1000:].high))
        support = mins[1][1]
        resistance = maxs[1][1]
        print("Current Support : ", support, " Will buy once " + symbol.upper() + " reaches this number.")
        print("Current Resistance : ", resistance)
        minimaIdxs, maximaIdxs = trendln.get_extrema((hist[-1000:].low, hist[-1000:].high))
        fig = trendln.plot_sup_res_date((hist[-1000:].low, hist[-1000:].high), hist[-1000:].index)
        fig.canvas.set_window_title(symbol.upper() + " Bot")
        fig.suptitle(symbol.upper() + " Support/Resistance Lines")
        plt.draw()
    except Exception as e:
        print('')
#On Bar Update
def run(sc):
    global hist
    global enteredTrade
    global symbol
    global timeframe
    global period
    global s
    hist = pd.DataFrame(hist)
    try:
        #Get current low and high
        low = hist.iloc[len(hist) - 1,2]
        high = hist.iloc[len(hist) - 1,1]
        if(low > 0):
            #Buy at support
            if (low <= support and not enteredTrade):
                order = wb.place_order(stock=symbol.upper(), action='BUY', orderType='MKT', enforce='DAY', quant=1)
                print(order)
                enteredTrade = True
            #Sell at resistance
            if (high >= resistance and enteredTrade):
                order = wb.place_order(stock=symbol.upper(), action='SELL', orderType='MKT', enforce='DAY', quant=1)
                print(order)
                enteredTrade = False
            #Update chart with new data
            hist = wb.get_bars(stock=symbol.upper(), interval='m'+timeframe, count=int((390*int(period))/int(timeframe)), extendTrading=0)
            hist = pd.DataFrame(hist)
            #call this method again every minute for new price changes
            drawChart(hist, True)
    except Exception as e:
        print(str(e))
    s.enter(60, 1, run, (sc,))
    plt.pause(60)

In [16]:
conn = StreamConn(debug_flg=False)
if not loginInfo['accessToken'] is None and len(loginInfo['accessToken']) > 1:
    conn.connect(loginInfo['uuid'], access_token=loginInfo['accessToken'])
else:
    conn.connect(wb.did)
#Initiate our scheduler so we can keep checking every minute for new price changes
s.enter(1, 1, run, (s,))
def get_data():
    global symbol
    global timeframe
    global period
    global hist
    try:
        #Symbol to trade
        symbol = input("Enter the symbol in uppercase letters you want to trade : ")
        print("Streaming real-time data now for " + symbol.upper())
        #Timeframe for candlesticks
        timeframe = input("Enter the timeframe in seconds to trade on (e.g. 1,5,15,60) : ")
        #Period for support / resistance calculation
        period = input("What is the period in hours do you want to use to calculate support/resistance (e.g. 1,5,30)  : ")
        #Get enough bars for the period, 390 minutes in 1 trading day multiplied by the period
        hist = wb.get_bars(stock=symbol.upper(), interval='s'+timeframe, count=int((390/60*int(period))/int(timeframe)), extendTrading=0)
        hist = pd.DataFrame(hist)
        s.run()
    except KeyboardInterrupt:
        print('Interrupted')
        pass
    except Exception as e:
        print("Make sure your timeframes are numbers (e.g. 1,5,15). Please try again.")
        get_data()
get_data()

say hello
Enter the symbol in uppercase letters you want to trade : gme
Streaming real-time data now for GME
Enter the timeframe in seconds to trade on (e.g. 1,5,15,60) : 60
What is the period in hours do you want to use to calculate support/resistance (e.g. 1,5,30)  : 1
Make sure your timeframes are numbers (e.g. 1,5,15). Please try again.
Enter the symbol in uppercase letters you want to trade : 1
Streaming real-time data now for 1
Enter the timeframe in seconds to trade on (e.g. 1,5,15,60) : 5
What is the period in hours do you want to use to calculate support/resistance (e.g. 1,5,30)  : 2

Interrupted


In [7]:
hists = {}
path = './daily_stock_data/'
for file in [f for f in listdir(path) if f.endswith('.csv')]:
    hists[file[:-4]] = pd.read_csv(path+file, index_col='time')

ValueError: Index time invalid

In [44]:
matplotlib.use('agg')

In [2]:
mc = fplt.make_marketcolors(
                        up='tab:blue',down='tab:red',
                        edge='black',
                        wick={'up':'blue','down':'red'},
                        volume='green',
                       )

s  = fplt.make_mpf_style(marketcolors=mc, mavcolors=["yellow","orange",'brown'])

for filename, hist in hists.items():
    print(filename)
    if not os.path.exists(f'./stock_graphs/{filename}/'):
        os.makedirs(f'./stock_graphs/{filename}/')
    for i in range(13,len(hist)-30):
        fplt.plot(
            hist.iloc[i:i+32],
            type='candle',
            style=s,
            volume=True,
            axisoff=True,
            mav=(5,8,13),
#             figscale=1.5,
            figsize=(32,32),
            xlim=(13, 45),
            savefig=f'./stock_graphs/{filename}/{filename}_{i}.jpeg'
        )

NameError: name 'fplt' is not defined

In [93]:
hist

,timestamp,open,high,low,close,volume,vwap,label
0,2021-02-18 09:31:00-05:00,3.760,3.800,3.760,3.800,49399.0,3.777,nothing
1,2021-02-18 09:32:00-05:00,3.763,3.765,3.750,3.765,12955.0,3.771,nothing
2,2021-02-18 09:33:00-05:00,3.750,3.750,3.750,3.750,955.0,3.771,nothing
3,2021-02-18 09:34:00-05:00,3.760,3.780,3.750,3.750,31990.0,3.767,nothing
4,2021-02-18 09:35:00-05:00,3.740,3.750,3.720,3.745,10369.0,3.762,buy
5,2021-02-18 09:36:00-05:00,3.765,3.790,3.765,3.780,7289.0,3.764,nothing
6,2021-02-18 09:37:00-05:00,3.780,3.794,3.774,3.794,3369.0,3.764,sell
7,2021-02-18 09:38:00-05:00,3.790,3.790,3.790,3.790,408.0,3.764,nothing
8,2021-02-18 09:39:00-05:00,3.770,3.780,3.770,3.780,2823.0,3.764,nothing
9,2021-02-18 09:40:00-05:00,3.780,3.783,3.770,3.770,11688.0,3.765,nothing
